In [3]:
import pandas as pd

# Load the dataset
file_path = "framinghamdataset.csv"
df = pd.read_csv(file_path)

# Check the number of rows
num_rows = len(df)

# Determine the split size
split_size = num_rows // 4

# Split the dataset into four equal parts
df1 = df.iloc[:split_size]
df2 = df.iloc[split_size:2*split_size]
df3 = df.iloc[2*split_size:3*split_size]
df4 = df.iloc[3*split_size:]

# Save the split datasets if needed
df1.to_csv("framingham_part1.csv", index=False)
df2.to_csv("framingham_part2.csv", index=False)
df3.to_csv("framingham_part3.csv", index=False)
df4.to_csv("framingham_part4.csv", index=False)

print("Datasets have been split and saved.")


Datasets have been split and saved.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import flwr as fl

# Define the MLP model class
class MLPModel(nn.Module):
    def __init__(self, input_size, hidden_units, dropout):
        super(MLPModel, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_units[0]),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_units[0], hidden_units[1]),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_units[1], 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)

# Load dataset function
def load_data(file_path):
    data = pd.read_csv(file_path)
    X = torch.tensor(data.drop('TenYearCHD', axis=1).values, dtype=torch.float32)
    y = torch.tensor(data['TenYearCHD'].values, dtype=torch.float32).unsqueeze(1)
    return DataLoader(TensorDataset(X, y), batch_size=32, shuffle=True)

# Federated Learning Client class
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, model, train_loader, client_id):
        self.model = model
        self.train_loader = train_loader
        self.client_id = client_id
        self.criterion = nn.BCELoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.0005)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

    def get_parameters(self, config):
        return [val.cpu().numpy() for _, val in self.model.state_dict().items()]

    def set_parameters(self, parameters):
        params_dict = zip(self.model.state_dict().keys(), parameters)
        state_dict = {key: torch.tensor(val) for key, val in params_dict}
        self.model.load_state_dict(state_dict, strict=True)

    def train(self):
        print(f"Client {self.client_id}: Training started")
        self.model.train()
        for data, target in self.train_loader:
            data, target = data.to(self.device), target.to(self.device)
            self.optimizer.zero_grad()
            output = self.model(data)
            loss = self.criterion(output, target)
            loss.backward()
            self.optimizer.step()
        print(f"Client {self.client_id}: Training completed")

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        self.model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for data, target in self.train_loader:
                data, target = data.to(self.device), target.to(self.device)
                output = self.model(data)
                predicted = (output > 0.5).float()
                total += target.size(0)
                correct += (predicted == target).sum().item()
        accuracy = correct / total
        print(f"Client {self.client_id}: Evaluation Accuracy = {accuracy:.4f}")
        return float(accuracy), len(self.train_loader.dataset), {"accuracy": accuracy}

# Initialize and start clients
def start_client(client_id, file_path):
    input_size = 15
    hidden_units = (128, 64)
    dropout = 0.3
    model = MLPModel(input_size, hidden_units, dropout)
    train_loader = load_data(file_path)
    client = FlowerClient(model, train_loader, client_id)
    print(f"Client {client_id}: Connecting to server...")
    fl.client.start_numpy_client(server_address="0.0.0.0:8080", client=client)

if __name__ == "__main__":
    import sys
    client_id = int(sys.argv[1])
    file_paths = {"framingham_part3.csv" 
    }
    start_client(client_id, file_paths[client_id])